# Build a RAG solution using Azure AI Search

Steps in this notebook:
1. Create an index 
2. Create a data source
3. Create a skillset
4. Create an indexer 
5. Send a query to the search engine to check results
6. Send query results to a language model to generate response

Note: Steps 1-4: Done during initial setup of Search Index only

## Install dependencies

In [1]:
%pip install python-dotenv
%pip install azure-core
%pip install azure-search-documents
%pip install azure-storage-blob
%pip install azure-identity
%pip install openai
%pip install aiohttp
%pip install ipywidgets
%pip install ipykernel


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 53.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Load Azure configurations

You always need to run this!

In [2]:
from dotenv import load_dotenv
import os

load_dotenv() # take environment variables from .env.

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT")
azure_openai_embeddings_deployment = os.getenv("AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT")
azure_openai_api_version = "2024-10-01-preview"
azure_search_service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
azure_search_service_admin_key = os.getenv("AZURE_SEARCH_ADMIN_KEY")
azure_search_service_index_name = "az-search-index-001"
azure_storage_connection_string = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
azure_ai_services_key = os.getenv("AZURE_AI_MULTISERVICE_KEY")

## Create an index

In [3]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchField,
    SearchFieldDataType,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    AzureOpenAIVectorizer,
    AzureOpenAIVectorizerParameters,
    SearchIndex,
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField,
    SemanticSearch,
    ScoringProfile,
    TagScoringFunction,
    TagScoringParameters
)

# Get credential from Azure AI Search Admin key
credential = AzureKeyCredential(azure_search_service_admin_key)

# Search index name  
index_name = azure_search_service_index_name

# Create a Search Index Client
index_client = SearchIndexClient(endpoint=azure_search_service_endpoint, credential=credential)

# Define the fields collection
fields = [
    SearchField(name="parent_id", type=SearchFieldDataType.String),  
    SearchField(name="title", type=SearchFieldDataType.String),
    SearchField(name="locations", type=SearchFieldDataType.Collection(SearchFieldDataType.String), filterable=True),
    SearchField(name="chunk_id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True, analyzer_name="keyword"),  
    SearchField(name="chunk", type=SearchFieldDataType.String, sortable=False, filterable=False, facetable=False),  
    SearchField(name="text_vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single), vector_search_dimensions=1024, vector_search_profile_name="myHnswProfile")
    ]  
  
# Configure the vector search configuration  
vector_search = VectorSearch(  
    algorithms=[  
        HnswAlgorithmConfiguration(name="myHnsw"),
    ],  
    profiles=[  
        VectorSearchProfile(  
            name="myHnswProfile",  
            algorithm_configuration_name="myHnsw",  
            vectorizer_name="myOpenAI",  
        )
    ],  
    vectorizers=[   # a vectorizer is software that performs vectorization
        AzureOpenAIVectorizer(  
            vectorizer_name="myOpenAI",  
            kind="azureOpenAI",  
            parameters=AzureOpenAIVectorizerParameters(  
                resource_url=azure_openai_endpoint,  
                deployment_name=azure_openai_embeddings_deployment,
                model_name=azure_openai_embeddings_deployment
            ),
        ),  
    ], 
)  

# New semantic configuration
semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="title"),
        keywords_fields=[SemanticField(field_name="locations")],
        content_fields=[SemanticField(field_name="chunk")]
    )
)

# Create the semantic settings with the configuration
semantic_search = SemanticSearch(configurations=[semantic_config])

# New scoring profile
scoring_profiles = [  
    ScoringProfile(  
        name="my-scoring-profile",
        functions=[
            TagScoringFunction(  
                field_name="locations",  
                boost=5.0,  
                parameters=TagScoringParameters(  
                    tags_parameter="tags",  
                ),  
            ) 
        ]
    )
]

# Create the search index
index = SearchIndex(name=index_name, 
                    fields=fields, 
                    vector_search=vector_search,
                    semantic_search=semantic_search,
                    scoring_profiles=scoring_profiles)  
result = index_client.create_or_update_index(index)  
print(f"{result.name} created")

az-search-index-001 created


## Check Azure Storage access
Verify access to storage and print out the documents

In [4]:
from azure.storage.blob import BlobServiceClient

# Initialize the BlobServiceClient with the connection string
blob_service_client = BlobServiceClient.from_connection_string(azure_storage_connection_string)

# Get the container client
container_client = blob_service_client.get_container_client("nasa-ebooks-pdfs-all")

# List blobs in the container
try:
    blobs_list = container_client.list_blobs()
    print("Blobs in the container:")
    for blob in blobs_list:
        print(blob.name)
    print("Access to the blob storage was granted.")
except Exception as e:
    print(f"Failed to access the blob storage: {e}")

Blobs in the container:
page-101.pdf
page-103.pdf
page-105.pdf
page-107.pdf
page-109.pdf
page-11.pdf
page-111.pdf
page-115.pdf
page-117.pdf
page-119.pdf
page-121.pdf
page-125.pdf
Access to the blob storage was granted.


## Create a Data Source

In [5]:
from azure.search.documents.indexes import SearchIndexerClient
from azure.search.documents.indexes.models import (
    SearchIndexerDataContainer,
    SearchIndexerDataSourceConnection
)

# Create a data source 
indexer_client = SearchIndexerClient(endpoint=azure_search_service_endpoint, credential=credential)
container = SearchIndexerDataContainer(name="nasa-ebooks-pdfs-all")
data_source_connection = SearchIndexerDataSourceConnection(
    name="ai-search-ds",
    type="azureblob",
    connection_string=azure_storage_connection_string,
    container=container
)
data_source = indexer_client.create_or_update_data_source_connection(data_source_connection)

print(f"Data source '{data_source.name}' created or updated")

Data source 'ai-search-ds' created or updated


## Create a Skillset

In [6]:
from azure.search.documents.indexes.models import (
    SplitSkill,
    InputFieldMappingEntry,
    OutputFieldMappingEntry,
    AzureOpenAIEmbeddingSkill,
    EntityRecognitionSkill,
    SearchIndexerIndexProjection,
    SearchIndexerIndexProjectionSelector,
    SearchIndexerIndexProjectionsParameters,
    IndexProjectionMode,
    SearchIndexerSkillset,
    CognitiveServicesAccountKey
)

# Create a skillset  
skillset_name = "ai-search-ss"

split_skill = SplitSkill(  
    description="Split skill to chunk documents",  
    text_split_mode="pages",  
    context="/document",  
    maximum_page_length=2000,  
    page_overlap_length=500,  
    inputs=[  
        InputFieldMappingEntry(name="text", source="/document/content"),  
    ],  
    outputs=[  
        OutputFieldMappingEntry(name="textItems", target_name="pages")  
    ],  
)  
  
embedding_skill = AzureOpenAIEmbeddingSkill(  
    description="Skill to generate embeddings via Azure OpenAI",  
    context="/document/pages/*",  
    resource_url=azure_openai_endpoint,  
    deployment_name=azure_openai_embeddings_deployment,  
    model_name=azure_openai_embeddings_deployment,
    dimensions=1024,
    inputs=[  
        InputFieldMappingEntry(name="text", source="/document/pages/*"),  
    ],  
    outputs=[  
        OutputFieldMappingEntry(name="embedding", target_name="text_vector")  
    ],  
)

entity_skill = EntityRecognitionSkill(
    description="Skill to recognize entities in text",
    context="/document/pages/*",
    categories=["Location"],
    default_language_code="en",
    inputs=[
        InputFieldMappingEntry(name="text", source="/document/pages/*")
    ],
    outputs=[
        OutputFieldMappingEntry(name="locations", target_name="locations")
    ]
)
  
index_projections = SearchIndexerIndexProjection(  
    selectors=[  
        SearchIndexerIndexProjectionSelector(  
            target_index_name=azure_search_service_index_name,  
            parent_key_field_name="parent_id",  
            source_context="/document/pages/*",  
            mappings=[  
                InputFieldMappingEntry(name="chunk", source="/document/pages/*"),  
                InputFieldMappingEntry(name="text_vector", source="/document/pages/*/text_vector"),
                InputFieldMappingEntry(name="locations", source="/document/pages/*/locations"),  
                InputFieldMappingEntry(name="title", source="/document/metadata_storage_name"),  
            ],  
        ),  
    ],  
    parameters=SearchIndexerIndexProjectionsParameters(  
        projection_mode=IndexProjectionMode.SKIP_INDEXING_PARENT_DOCUMENTS  
    ),  
) 

cognitive_services_account = CognitiveServicesAccountKey(key=azure_ai_services_key)

skills = [split_skill, embedding_skill, entity_skill]

skillset = SearchIndexerSkillset(  
    name=skillset_name,  
    description="Skillset to chunk documents, generate embeddings, and extract location entities",  
    skills=skills,  
    index_projection=index_projections,
    cognitive_services_account=cognitive_services_account
)
  
client = SearchIndexerClient(endpoint=azure_search_service_endpoint, credential=credential)  
client.create_or_update_skillset(skillset)  
print(f"{skillset.name} created")  

ai-search-ss created


## Create an Indexer

In [7]:
from azure.search.documents.indexes.models import (
    SearchIndexer,
    IndexingSchedule
)

# Create an indexer  
indexer_name = "ai-search-idxr" 

# Schedule to run every 24 hours
schedule = IndexingSchedule(interval="P1D")

indexer_parameters = None

indexer = SearchIndexer(  
    name=indexer_name,  
    description="Indexer to index documents, generate embeddings, and extract entities",  
    skillset_name=skillset_name,  
    target_index_name=index_name,  
    data_source_name=data_source.name,
    parameters=indexer_parameters,
    schedule=schedule
)  

# Create and run the indexer  
indexer_client = SearchIndexerClient(endpoint=azure_search_service_endpoint, credential=credential)  
indexer_result = indexer_client.create_or_update_indexer(indexer)  

print(f' {indexer_name} is created and running. Give the indexer a few minutes before running a query.')  

 ai-search-idxr is created and running. Give the indexer a few minutes before running a query.


## Send a query to the search engine to check results

Executed every time a user makes a query

## Perform a Hybrid Search

In [8]:
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery
from azure.core.credentials import AzureKeyCredential

# Get credential from Azure AI Search Admin key
credential = AzureKeyCredential(azure_search_service_admin_key)
search_client = SearchClient(endpoint=azure_search_service_endpoint, 
                             credential=credential, 
                             index_name=azure_search_service_index_name)

# User Query
query = "What can I see in the United States?"  

# Convert query into vector form
vector_query = VectorizableTextQuery(text=query, 
                                     k_nearest_neighbors=50, 
                                     fields="text_vector",
                                     weight=1)

results = search_client.search(  
    search_text=query,  #This performs a full-text search using the query
    vector_queries= [vector_query], #This adds a vector search component
    select=["title","chunk","locations"], #Specify fields to be return in the result
    top=3
) 

for result in results:  
    print(f"Score: {result['@search.score']} \n")
    print(f"Title: {result['title']} \n")
    print(f"Chunk: {result['chunk']} \n")
    print(f"Locations: {result['locations']}")


Score: 0.03306011110544205 

Title: page-103.pdf 

Chunk: L
a

n
d

E
A

R
T

H

96

Bowknot Bend
United States

This section of the Green River canyon in eastern Utah is known as Bowknot Bend because of the way the river doubles back on 

itself. The loop carries river rafters 14.5 kilometers (9 miles) before bringing them back to nearly the same point they started from—

though on the other side of a low, narrow saddle. The reason for the tight bends in the Green River is the same as it is for the mighty 

Mississippi: river courses often wind over time when they flow across a bed of relatively soft sediment in a floodplain.

In this January 2014 photograph taken from the International Space Station, the Green River appears dark because it lies in deep 

shadow, 300 meters (1,000 feet) below the surrounding landscape. The yellow-tinged cliffs that face the rising Sun give a sense of 

the steep canyon walls. The straight white line across the scene is the contrail from a jetliner. 



## Perform a Semantic Hybrid Search

In [9]:
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery
from azure.core.credentials import AzureKeyCredential

# Get credential from Azure AI Search Admin key
credential = AzureKeyCredential(azure_search_service_admin_key)
search_client = SearchClient(endpoint=azure_search_service_endpoint, 
                             credential=credential, 
                             index_name=azure_search_service_index_name)

# User Query
query = "What can I see in the United States?"  

# Convert query into vector form
vector_query = VectorizableTextQuery(text=query, 
                                     k_nearest_neighbors=50, 
                                     fields="text_vector",
                                     weight=1)

results = search_client.search(
    query_type="semantic", 
    semantic_configuration_name='my-semantic-config',
    scoring_profile="my-scoring-profile",
    scoring_parameters=["tags-beach, 'United States'"], 
    search_text=query,
    vector_queries= [vector_query],
    select=["title","chunk","locations"],
    top=3,
)

# Sort the results by score in descending order
sorted_results = sorted(results, key=lambda x: x['@search.score'], reverse=True)

for result in sorted_results:  
    print(f"Score: {result['@search.score']} \n")
    print(f"Title: {result['title']} \n")
    print(f"Chunk: {result['chunk']} \n")
    print(f"Locations: {result['locations']}")



Score: 0.3306010961532593 

Title: page-105.pdf 

Chunk: L
a

n
d

E
A

R
T

H

98

From Rainforest to Rain Shadow
United States

Within a three-hour drive across Oregon, you can visit a beach, a temperate rainforest, a mountain glacier, and the high desert.  

The diversity of the landscape is mostly driven by the interaction of air masses and mountains.

This false-color Landsat 5 image from October 2011 shows the bare soil and sparse vegetation of the high desert in shades of pink, 

together with the deep-green vegetation on the west side of the Cascade Mountains. The one blue spot is the glacial cap of Mount Hood. 

The transition from green to brown is indicative of a “rain shadow.” Winds blowing from the west carry moisture from the Pacific 

Ocean. As the air moves up into the mountains, it cools and the pressure decreases; the moisture condenses and falls out as rain or 

snow. On the eastern side, as the elevation drops, the air pressure increases and the air warms, effective

## Send query results to a language model to generate response

In [10]:
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery
from azure.core.credentials import AzureKeyCredential
from openai import AzureOpenAI

# Get credential from Azure AI Search Admin key
credential = AzureKeyCredential(azure_search_service_admin_key)
search_client = SearchClient(endpoint=azure_search_service_endpoint, 
                             credential=credential, 
                             index_name=azure_search_service_index_name)

# Azure OpenAI client
openai_client = AzureOpenAI(
    # to get version: https://learn.microsoft.com/en-us/azure/ai-services/openai/api-version-deprecation
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    api_key=azure_openai_key)

# Provide instructions to the model
SYSTEM_PROMPT="""
You are an AI assistant that helps users learn from the information found in the source material.
Answer the query using only the sources provided below.
Use bullets if the answer has multiple points.
If the answer is longer than 3 sentences, provide a summary.
Answer ONLY with the facts listed in the list of sources below. Cite your source when you answer the question
If there isn't enough information below, say you don't know.
Do not generate answers that don't use the sources below.
Query: {query}
Sources:\n{sources}
"""

# User Query
query = "What can I see in the United States?"  

# Convert query into vector form
vector_query = VectorizableTextQuery(text=query, 
                                     k_nearest_neighbors=50, 
                                     fields="text_vector",
                                     weight=1)

results = search_client.search(
    query_type="semantic", 
    semantic_configuration_name='my-semantic-config',
    search_text=query,
    vector_queries= [vector_query],
    select=["title","chunk","locations"],
    top=5,
)

# Use a unique separator to make the sources distinct. 
# We chose repeated equal signs (=) followed by a newline because it's unlikely the source documents contain this sequence.
sources_formatted = "=================\n".join([f'TITLE: {document["title"]}, CONTENT: {document["chunk"]}, LOCATIONS: {document["locations"]}' for document in results])

response = openai_client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": SYSTEM_PROMPT.format(query=query, sources=sources_formatted)
        }
    ],
    model=azure_openai_deployment
)

print(response.choices[0].message.content)


You can see the following in the United States:

- In Oregon, within a three-hour drive, you can visit different landscapes including a beach, temperate rainforest, mountain glacier, and high desert. The landscape diversity is influenced by air masses and mountain interactions, leading to varied climatic conditions such as the "rain shadow" effect. Notable sites include the deep-green vegetation on the west side of the Cascade Mountains and Mount Hood's glacial cap. (Source: page-105.pdf)

- In eastern Utah, you can visit Bowknot Bend on the Green River. This area is known for its unique river loop where the river doubles back on itself, creating a natural attraction for river rafters. The surrounding scenery includes deep canyons and steep, yellow-tinged cliffs. (Source: page-103.pdf)
